# <font color='purple' face='segoe ui'>LIVE IMAGE CLASSIFIER + MOTOR CONTROLLER</font>
<br>
#### A 2nd Generation Model of the Image Classifier <br>

#### Takes a picture from Raspberry Pi Camera and performs live image classification

In [ ]:
import numpy as np #math
import keras #machine learning
import urllib,urllib.request
import cv2
import time, os, paramiko
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from resizeimage import resizeimage

In [ ]:
datagen = ImageDataGenerator(
       rotation_range=40,
       width_shift_range=0.2,
       height_shift_range=0.2,
       shear_range=0.2,
       zoom_range=0.2,
       horizontal_flip=True,
       fill_mode='nearest')

img = load_img('data/train/permit/image(1).png')  # this is a PIL image
img = img.resize((150,150))
x = img_to_array(img)  # this is a Numpy array with shape (150, 150, 3)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 150, 150, 3)
# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in datagen.flow(x, batch_size=1,
                         save_to_dir='preview', save_prefix='permit', save_format='jpg'):
   i += 1
   if i > 20:
       break  # otherwise the generator would loop indefinitely 


model1 = Sequential()
model1.add(Conv2D(32, (3, 3), input_shape=(150,150, 3)))
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))

model1.add(Conv2D(32, (3, 3)))
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))

model1.add(Conv2D(64, (3, 3)))
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))

# In[4]:

model1.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model1.add(Dense(64))
model1.add(Activation('relu'))
model1.add(Dropout(0.5))
model1.add(Dense(1))
model1.add(Activation('sigmoid'))

model1.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


In [ ]:
model1.load_weights('Weights/second_try.h5')

In [ ]:
def make_prediction1(the_image):
    
    img = resizeimage.resize_cover(the_image, [150, 150])
    new_img = img_to_array(img)
    fin = new_img.reshape((1,) + new_img.shape)
    prediction = model1.predict(fin)
    the_prediction = prediction[0][0]
    
    if the_prediction == 1.0:
        
        return "PERMIT"
    
    else:
        return "DENY"

In [ ]:
def identify():
    
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect('192.168.1.3', username='pi', password='raspberry', port=22)
    channel = ssh.invoke_shell()
    out = channel.recv(9999)

    channel.send("raspistill -o image.jpg -t 1400\n")
    sftp = ssh.open_sftp()
    sftp.get('image.jpg','C:\\Users\\Mint\\Documents\\secure_gun\\image.jpg',callback=None)
    sftp.get('image.jpg','C:\\Users\\Mint\\Documents\\secure_gun\\image.jpg',callback=None)
    sftp.get('image.jpg','C:\\Users\\Mint\\Documents\\secure_gun\\image.jpg',callback=None)
    sftp.get('image.jpg','C:\\Users\\Mint\\Documents\\secure_gun\\image.jpg',callback=None)
    sftp.close()
    img = cv2.imread("image.jpg")
    image = Image.fromarray(img,'RGB')

    final_pred = make_prediction1(image)
    image.show()
    os.remove('image.jpg')
    if final_pred == 'PERMIT':
        
        channel.send('cd project\n')
        channel.send('python motorcontrol.py\n')
        time.sleep(7)
        channel.send('python motorcontrol2.py\n')
        return final_pred
    else:
        return final_pred

In [ ]:
print(identify())